In [333]:
!pip install wikipedia-api faiss-cpu sentence-transformers transformers datasets nltk rouge-score cohere google-api-python-client rake-nltk

In [334]:
import wikipediaapi
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import cohere
import os
import spacy
from googleapiclient.discovery import build
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [335]:
# Налаштування Wikipedia API
wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='MyNLPApp/1.0 (myemail@example.com)'
)

# Налаштування Google Search API
api_key = "MY API KEY"
cse_id = "MY CSE ID"

def google_search(query, api_key, cse_id, num_results=1):
    """
    Выполняет поиск в Google и возвращает результаты.
    """
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
    return [item['snippet'] for item in res.get('items', [])]

# Завантажуємо модель SpaCy для визначення сутностей
nlp = spacy.load("en_core_web_sm")

def extract_main_topics(context):
    """
    Витягує ключові сутності та ключові слова з тексту для використання в запиті до Wikipedia або Google.
    """
    doc = nlp(context)
    topics = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "LOC", "EVENT", "WORK_OF_ART"]]

    # Використовуємо RAKE для виділення ключових слів
    rake = Rake()
    rake.extract_keywords_from_text(context)
    keywords = rake.get_ranked_phrases()

    # Об'єднуємо сутності та ключові слова, щоб отримати розширений набір тем
    topics.extend(keywords)

    # Видаляємо дублікати та залишаємо унікальні теми
    topics = list(set(topics))

    # Якщо досі немає тем, використовуємо весь контекст як запит
    if not topics:
        topics = [context]

    return topics

def fetch_wikipedia_article(query):
    """
    Отримує текст статті з Wikipedia на основі запиту.
    """
    page = wiki.page(query)
    if page.exists():
        return page.summary[:500]  # Обмежуємо текст до 500 символів
    return None

def get_relevant_content(context):
    """
    Знаходить статті в Wikipedia або Google, релевантні основним темам контексту.
    Повертає список знайдених статей.
    """
    topics = extract_main_topics(context)
    content = []
    for topic in topics:
        wikipedia_content = fetch_wikipedia_article(topic)
        if wikipedia_content:
            content.append(wikipedia_content)
        google_results = google_search(topic, api_key, cse_id, num_results=5)
        for google_result in google_results:
            content.append(google_result[:500])
    # Також додаємо результати пошуку за всім контекстом
    google_results_full_context = google_search(context, api_key, cse_id, num_results=5)
    for google_result in google_results_full_context:
        content.append(google_result[:500])
    return content

In [336]:
# Завантажуємо датасет CNN/Daily Mail
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:100]")  # Використовуємо перші 100 текстів для тесту

# Створюємо корпус з перших 100 статей
cnn_corpus = [item['article'][:500] for item in dataset]

# Об'єднуємо статті з CNN/Daily Mail
full_corpus = cnn_corpus

In [337]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def prepare_faiss_index(corpus):
    embeddings = embedder.encode(corpus, convert_to_tensor=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.cpu().numpy())
    return index, corpus

# Підготовка початкового індексу
index, full_corpus = prepare_faiss_index(full_corpus)


In [338]:
def update_corpus_with_content(context, corpus, index):
    """
    Витягує статті з Wikipedia або Google на основі ключових тем контексту і додає їх у корпус.
    Кожну знайдену статтю додає окремо.
    """
    relevant_content_list = get_relevant_content(context)
    for relevant_content in relevant_content_list:
        if relevant_content != "No relevant information found.":
            if relevant_content not in corpus:
                corpus.append(relevant_content)
                index.add(embedder.encode([relevant_content]))
    return corpus

def retrieve_documents(context, index, corpus, k=10):
    """
    Витягує топ-K релевантних документів і скорочує їх до заданої кількості слів.
    """
    context_embedding = embedder.encode(context, convert_to_tensor=True).numpy()
    distances, indices = index.search(context_embedding.reshape(1, -1), k)
    retrieved_docs = [corpus[i] for i in indices[0]]
    # Скорочуємо кожен документ до 100 слів
    shortened_docs = ["".join(doc[:500]) for doc in retrieved_docs]
    return shortened_docs

In [339]:
# Налаштування API ключа для Cohere
co = cohere.ClientV2('Cohere API-KEY')

def generate_completion_cohere(context, retrieved_info=None):
    """
    Генерація тексту на основі контексту і витягнутої інформації (якщо є) з використанням Cohere.
    """
    if retrieved_info:
        prompt = f"Additional information: {retrieved_info}\nComplete the text:{context}\n"
    else:
        prompt = f"Complete the text: {context}\n"

    print(f"Prompt: {prompt}")
    response = co.chat(
        model="command-r",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return response.message.content[0].text

In [340]:
def rag_text_completion(context, index, corpus):
    """
    Повний пайплайн RAG для доповнення тексту.
    """
    # Оновлення корпусу з Wikipedia або Google
    corpus = update_corpus_with_content(context, corpus, index)

    # Витягування релевантної інформації
    retrieved_docs = retrieve_documents(context, index, corpus, k=3)
    retrieved_info = ";".join(retrieved_docs)

    # Генерація тексту з використанням RAG
    completed_text = generate_completion_cohere(context, retrieved_info)
    return completed_text

In [341]:
context = "US Dollar to Ukranian Hryvnia course is : "

# Генерація тексту без використання RAG
generated_text_without_rag = generate_completion_cohere(context)

# Генерація тексту з використанням RAG
generated_text_with_rag = rag_text_completion(context, index, full_corpus)

documents = retrieve_documents(context,index,full_corpus)

for document in documents:
    print(document)

print("Generated Text without RAG:")
print(generated_text_without_rag)
print("\nGenerated Text with RAG:")
print(generated_text_with_rag)

Prompt: Complete the text: US Dollar to Ukranian Hryvnia course is : 

Prompt: Additional information: Convert US Dollar to Ukrainian Hryvnia. USD. UAH. 1 USD. 41.3354 UAH. 5 USD. 206.677 UAH. 10 USD. 413.354 UAH. 25 USD. 1,033.38 UAH. 50 USD. 2,066.77 UAH. 100 ...;Get the latest 1 US Dollar to Ukrainian Hryvnia rate for FREE with the original Universal Currency Converter. Set rate alerts for to and learn more about US ...;Get the latest 1 Ukrainian Hryvnia to US Dollar rate for FREE with the original Universal Currency Converter. Set rate alerts for to and learn more about ...
Complete the text:US Dollar to Ukranian Hryvnia course is : 

Convert US Dollar to Ukrainian Hryvnia. USD. UAH. 1 USD. 41.3354 UAH. 5 USD. 206.677 UAH. 10 USD. 413.354 UAH. 25 USD. 1,033.38 UAH. 50 USD. 2,066.77 UAH. 100 ...
Get the latest 1 US Dollar to Ukrainian Hryvnia rate for FREE with the original Universal Currency Converter. Set rate alerts for to and learn more about US ...
Get the latest 1 Ukrainian Hr

In [342]:
# Цільовий текст для порівняння
reference = "1 USD equals to 41.3354 UAH"

# Оцінка BLEU
bleu_without_rag = sentence_bleu([reference.split()], generated_text_without_rag.split())
bleu_with_rag = sentence_bleu([reference.split()], generated_text_with_rag.split())

print(f"BLEU Score without RAG: {bleu_without_rag}")
print(f"BLEU Score with RAG: {bleu_with_rag}")

# Оцінка ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge_without_rag = scorer.score(reference, generated_text_without_rag)
rouge_with_rag = scorer.score(reference, generated_text_with_rag)

print("\nROUGE Score without RAG:")
print(rouge_without_rag)
print("\nROUGE Score with RAG:")
print(rouge_with_rag)

BLEU Score without RAG: 1.1896457329133973e-231
BLEU Score with RAG: 9.85444998995587e-155

ROUGE Score without RAG:
{'rouge1': Score(precision=0.3333333333333333, recall=0.5714285714285714, fmeasure=0.4210526315789474), 'rougeL': Score(precision=0.3333333333333333, recall=0.5714285714285714, fmeasure=0.4210526315789474)}

ROUGE Score with RAG:
{'rouge1': Score(precision=0.75, recall=0.8571428571428571, fmeasure=0.7999999999999999), 'rougeL': Score(precision=0.75, recall=0.8571428571428571, fmeasure=0.7999999999999999)}
